In [20]:
import json
import torch
import faiss
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer

In [21]:
model = SentenceTransformer('all-MiniLM-L6-v2')

with open('conversas_podroes_filtrado.json', 'r', encoding='utf-8') as f:
    conversas = json.load(f)

mensagens = [msg["message"] for msg in conversas]

# Gerar embeddings para cada mensagem
embeddings = []
for mensagem in tqdm(mensagens, desc="Gerando embeddings"):
    emb = model.encode(mensagem, convert_to_tensor=False)
    embeddings.append(emb)

# Salvar os embeddings em um arquivo .npy
model.save("modelo_podrao")
print("Embeddings gerados e salvos com sucesso!")

Gerando embeddings: 100%|██████████| 9199/9199 [02:18<00:00, 66.21it/s]


Embeddings gerados e salvos com sucesso!


In [22]:
# Converter a lista de embeddings para um array NumPy
embeddings_np = np.array(embeddings)

# Definir a dimensão (no nosso caso, 384 para all-MiniLM-L6-v2)
dimension = embeddings_np.shape[1]

# Criar um índice FAISS (IndexFlatL2 é simples e sem custo)
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

faiss.write_index(index, "faiss_index.index")
print(f"Indexado {index.ntotal} vetores!")

Indexado 9199 vetores!


In [50]:
model = SentenceTransformer('modelo_podrao')

# Carregar o índice FAISS salvo
index = faiss.read_index("faiss_index.index")

# Carregar o JSON de conversas
with open('conversas_podroes_filtrado.json', 'r', encoding='utf-8') as f:
    conversas = json.load(f)

def recuperar_contexto(query, k=3):
    # Gerar embedding para a query
    query_embedding = model.encode([query], convert_to_tensor=False)
    query_embedding = np.array(query_embedding).astype("float32")
    
    # Buscar os k vetores mais próximos no índice FAISS
    distances, indices = index.search(query_embedding, k)
    
    # Recuperar as mensagens de contexto com base nos índices
    context_msgs = [conversas[i]["message"] for i in indices[0]]
    return context_msgs

query = "Filme chato pra caralho"
contexto = recuperar_contexto(query)
print("Resposta gerada:", contexto)

Resposta gerada: ['Caralho... esse filme é muito trash e muito velho!!!', 'Burro pra caralho', 'Estou no cinema pra assistir duna 2, devo jogar mais pra noite']
